In [ ]:
# install fastkaggle if not available
try: import fastkaggle
except ModuleNotFoundError:
    !pip install -Uq fastkaggle

from fastkaggle import *

## Getting set up

In [ ]:
comp = 'playground-series-s3e25'
path = setup_comp(comp, install='')

In [ ]:
path

Path('playground-series-s3e25')

In [ ]:
trn_path = path/'train.csv'

In [ ]:
import numpy as np, pandas as pd
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, FunctionTransformer, QuantileTransformer
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import make_scorer, mean_absolute_error, median_absolute_error as medae
from scipy.stats import loguniform

In [ ]:
df = pd.read_csv(trn_path)
X = df.iloc[:,1:-1] # drop id and target
y = df.iloc[:,-1]
X_tr, X_dev, y_tr, y_dev = train_test_split(X,y,test_size=0.2) # XXyy

In [ ]:
scaler = StandardScaler()
model = MLPRegressor(max_iter=4000, hidden_layer_sizes=(64,32,16))
pipe = make_pipeline(scaler,model)
pipe.fit(X_tr,y_tr)
medae(pipe.predict(X_dev), y_dev)

0.7331671176816572

In [ ]:
scaler = StandardScaler()
model = MLPRegressor(max_iter=4000, hidden_layer_sizes=(32,16))
pipe = make_pipeline(scaler,model)

pipe.named_steps['mlpregressor'].learning_rate_init = 0.013

from sklearn import set_config
set_config(transform_output='pandas')
nepoch=200
X_tr_tf=pipe[:-1].fit_transform(X_tr)
for i in range(nepoch):
    pipe[-1].learning_rate_init = 0.013*0.5**i
    pipe[-1].partial_fit(X_tr_tf,y_tr)

medae(pipe.predict(X_dev), y_dev)

## Submitting to Kaggle

In [ ]:
ss = pd.read_csv(path/'sample_submission.csv')

In [ ]:
tst = pd.read_csv(path/'test.csv')

In [ ]:
ss['Hardness'] = pipe.predict(tst.iloc[:,1:])

In [ ]:
ss.to_csv('subm.csv', index=False)
!head subm.csv

id,Hardness
10407,2.2447038862580264
10408,2.9816768410094707
10409,6.057493409630426
10410,4.0252023999737645
10411,6.180580306667615
10412,4.6798035133943365
10413,3.7570595169337735
10414,5.385486430590966
10415,3.173753209604238


In [ ]:
if not iskaggle:
    from kaggle import api
    api.competition_submit_cli('subm.csv', 'MLP', comp)

100%|███████████████████████████████████████████████████████| 164k/164k [00:00<00:00, 185kB/s]


## Conclusion

## Addendum

In [ ]:
if not iskaggle:
    push_notebook('xy', 'mock-first-steps-road-to-the-top-part-1',
                  title='mock First Steps: Road to the Top, Part 1',
                  file='mock-first-steps-road-to-the-top-part-1.ipynb',
                  competition=comp, private=False, gpu=True)